In [1]:
import pandas as pd
import numpy as np
import itertools
from itertools import combinations

In [2]:
order_products = pd.read_csv('data/order_products__prior.csv')[['order_id','product_id']]
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
aisles = pd.read_csv('data/aisles.csv')

In [3]:
order_products_subset = order_products
x = order_products_subset.merge(products[['product_id','aisle_id']], on = 'product_id')
market_baskets = x.groupby('order_id')['product_id'].apply(set)
orders = orders[['order_id','user_id']]

In [7]:
market_baskets_subset = market_baskets[:1000000]

In [52]:
def apriori(T, min_sup):
    frequency_table = {}
    L = find_frequent_1item(T, min_sup)
    frequency_table.update(L)

    #Keep pruning and finding frequent patterns until no more supersets can be formed
    while(L):
        c_k = apriori_gen(L)
        L = min_prune(find_occurrences(T, c_k), min_sup)
        frequency_table.update(L)
        
    return frequency_table

def apriori_gen(L):
    c_k = set()
    
    candidate_items = set()
    #Finds all combinations formed from List_k-1
    for i in L.keys():
        candidate_items = candidate_items.union(i)

    #Finds all combinations of size k itemsets from the candidate itemset
    candidates = list(itertools.combinations(candidate_items, len(i) + 1))

    #Check each combination to see if the subsets are infrequent
    for combo in candidates:
        if(not has_infrequent_subset(combo, L)):
            c_k.add(frozenset(combo))

    return c_k

def has_infrequent_subset(candidate, L):
    for subset in itertools.combinations(candidate, len(candidate) - 1):
        if(frozenset(subset) not in L):
            return True
    return False

def find_frequent_1item(T, min_sup):
    L1 = {}
    for row in T:
        for item in row:
            L1[frozenset([item])] = L1.get(frozenset([item]), 0) + 1

    return min_prune(L1, min_sup)

'''
Returns a dictionary with the occurrences in the database
'''
def find_occurrences(T, subsets):
    item_counts = {}

    for itemset in T:
        for subset in subsets:
            if subset.intersection(itemset) == subset:
                item_counts[frozenset(subset)] = item_counts.get(frozenset(subset), 0) + 1

    return item_counts

'''
Prunes items that do not meet the minimum support
'''
def min_prune(dict_k, min_sup):
    return {k:v for k,v in dict_k.items() if v >= min_sup}

In [53]:
import time

times = []
for support in [5000, 6000, 7000, 8000, 9000, 10000]:
    start = time.time()
    apriori(market_baskets_subset.to_list(), min_sup = support)
    end = time.time()
    times.append(end - start)

In [5]:
%load_ext Cython

In [16]:
%load_ext line_profiler

In [68]:
%%cython -f --compile-args=-DCYTHON_TRACE=1
cimport cython
import itertools

@cython.binding(True)
@cython.linetrace(True)   
def apriori(T, min_sup):
    frequency_table = {}
    L = find_frequent_1item(T, min_sup)
    frequency_table.update(L)

    #Keep pruning and finding frequent patterns until no more supersets can be formed
    while(L):
        c_k = apriori_gen(L)
        L = min_prune(find_occurrences(T, c_k), min_sup)
        frequency_table.update(L)
        
    return frequency_table

def apriori_gen(L):
    c_k = set()
    
    candidate_items = set()
    #Finds all combinations formed from List_k-1
    for i in L.keys():
        candidate_items = candidate_items.union(i)

    #Finds all combinations of size k itemsets from the candidate itemset
    candidates = list(itertools.combinations(candidate_items, len(i) + 1))

    #Check each combination to see if the subsets are infrequent
    for combo in candidates:
        if(not has_infrequent_subset(combo, L)):
            c_k.add(frozenset(combo))

    return c_k

def has_infrequent_subset(candidate, L):
    for subset in itertools.combinations(candidate, len(candidate) - 1):
        if(frozenset(subset) not in L):
            return True
    return False

def find_frequent_1item(T, min_sup):
    L1 = {}
    for row in T:
        for item in row:
            L1[frozenset([item])] = L1.get(frozenset([item]), 0) + 1

    return min_prune(L1, min_sup)

'''
Returns a dictionary with the occurrences in the database
'''
def find_occurrences(T, subsets):
    item_counts = {}

    for itemset in T:
        for subset in subsets:
            if subset.intersection(itemset) == subset:
                item_counts[frozenset(subset)] = item_counts.get(frozenset(subset), 0) + 1

    return item_counts

'''
Prunes items that do not meet the minimum support
'''
def min_prune(dict_k, min_sup):
    return {k:v for k,v in dict_k.items() if v >= min_sup}

In [70]:
import time

times = []
for support in [5000, 6000, 7000, 8000, 9000, 10000]:
    start = time.time()
    apriori(market_baskets_subset.to_list(), min_sup = support)
    end = time.time()
    times.append(end - start)

In [6]:
%%cython -f --compile-args=-DCYTHON_TRACE=1
cimport cython
import itertools

@cython.binding(True)
@cython.linetrace(True)   

class LinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    class Node:
        def __init__(self, data=None, next=None):
            self.data = data
            self.next = next

    def __repr__(self):
        return 'Node-LinkedList'

    def append(self, data):
        if self.head is None:
            self.head = self.Node(data, self.head)
            self.tail = self.head
        else:
            newNode = self.Node(data)
            self.tail.next = newNode
            self.tail = newNode

    def sumcounts(self):
        total = 0
        current = self.head
        while (current is not None):
            total += current.data.count
            current = current.next
        return total

    def printall(self):
        current = self.head
        while (current is not None):
            print(current.data.item, current.data.count)
            current = current.next
            
class TreeNode:
    def __init__(self, item = None, count=1):
        self.item = item
        self.count = count
        self.parent = None
        self.header = None
        self.next = []

    def get_pattern(self):
        collection = []
        current = self

        while current.parent is not None:
            collection.insert(0, current.parent.item)
            current = current.parent

        return collection

class FP_Tree:
    def __init__(self, header_table):
        self.root = TreeNode()
        self.header = header_table

    def insert_tree(self, itemset, header_table, count=1):
        current = self.root

        for item in itemset:
            found_match = False

            for index in range(0, len(current.next)):
                if item == current.next[index].item:
                    current = current.next[index]
                    current.count += count
                    found_match = True
                    break

            if item is None:
                continue
            if not found_match:
                newNode = TreeNode(item, count)
                newNode.parent = current
                header_table[item].append(newNode)
                current.next.append(newNode)
                current = newNode

    def print_tree(self):
        self.print_tree_wrapper(self.root, 0)

    def print_tree_wrapper(self, node, shift):
        if node is None:
            return
        for node in node.next:
            print(shift*"\t", node.item, node.count)
            self.print_tree_wrapper(node, shift+1)
            
            
'''
Construction of the fp-tree given the initial data. 
This step does the initial 1-itemset frequency prune
@Returns the fp-tree 
'''
def fp_construct(T, min_sup):
    L1 = {}
    for row in T:
        for item in row:
            L1[tuple([item])] = L1.get(tuple([item]), 0) + 1
    L = min_prune(L1, min_sup)
    L_desc = dict(sorted(L.items(), key=lambda item: item[1], reverse=True))
    ordered_keys = L_desc.keys()
    ordered_itemsets = [[],[]]

    #Create ordered itemsets
    for row in T:
        record = []
        for item in ordered_keys:
            if item[0] in row:
                record.append(item)
        ordered_itemsets[0].append(record)
        ordered_itemsets[1].append(1)

    return generate_tree(L_desc, ordered_itemsets)

'''
Generates an FP-tree given the ordered itemsets and items in descending frequency
@Returns the fp-tree
'''
def generate_tree(L_desc, ordered_itemsets):
    header_table = {}
    # Link each atomic item to a linkedlist
    for item in L_desc:
        linkedlist = LinkedList()
        header_table[item] = linkedlist

    newTree = FP_Tree(header_table)

    #ordered_itemsets[0] is the item. ordered_itemsets[1] is the counts
    for i in range(0, len(ordered_itemsets[0])):
        newTree.insert_tree(ordered_itemsets[0][i], header_table, ordered_itemsets[1][i])

    newTree.root.header = header_table
    return newTree

'''
Performs the actual mining of the data and spits it out into the global frequency itemset list
'''
def fp_growth(tree, alpha, min_sup, frequency_table):
    single_path = False
    current = tree.root

    while len(current.next) == 1:
        current = current.next[0]
        if current.next is None:
            single_path = True

    if single_path:
        current = tree.root
        combo_choices = {}

        while current.next[0] is not None:
            if current.next[0].count < min_sup:
                current.next[0] = None
            else:
                current = current.next[0]
                combo_choices[current.item] = current.count

        combo_list = []
        for i in range(1, len(combo_choices) + 1):
            combo_list += list(itertools.combinations(combo_choices, i))

        for combo in combo_list:
            count = combo_choices[combo[-1]] #Frequency is the count of the last node count
            itemset = alpha + list(combo)
            frequency_table[tuple(itemset)] = count

    else:
        for k, v in tree.header.items():
            if v.sumcounts() >= min_sup:
                beta = alpha + list(k)
                frequency_table[tuple(beta)] = v.sumcounts()
                ordered_itemsets = [[], []]
                L_desc = []

                current = v.head
                while current is not None:
                    ordered_itemsets[0].append(current.data.get_pattern())
                    ordered_itemsets[1].append(current.data.count)
                    current = current.next

                for key in tree.header.keys():
                    L_desc.append(key)
                L_desc.remove(k)

                newTree = generate_tree(L_desc, ordered_itemsets)
                if tree is not None:
                    fp_growth(newTree, beta, min_sup, frequency_table)
                    
def min_prune(dict_k, min_sup):
    return {k:v for k,v in dict_k.items() if v >= min_sup}

In [93]:
import time

times = []
for support in [5000, 6000, 7000, 8000, 9000, 10000]:
    start = time.time()
    
    frequency_table = {}
    min_support = support

    tree = fp_construct(market_baskets_subset, min_support)
    fp_growth(tree, [], min_support, frequency_table)

    end = time.time()
    times.append(end - start)

In [10]:
frequency_table = {}
min_support = int(0.001*len(market_baskets_subset))

tree = fp_construct(market_baskets_subset, min_support)
fp_growth(tree, [], min_support, frequency_table)

In [11]:
frequency_table

{(24852,): 147307,
 (13176,): 118215,
 (21137,): 82662,
 (21137, 24852): 17498,
 (21137, 13176): 19230,
 (21903,): 74894,
 (21903, 24852): 15906,
 (21903, 13176): 15585,
 (21903, 21137): 11781,
 (21903, 21137, 24852): 2923,
 (21903, 21137, 13176): 3329,
 (47209,): 66439,
 (47209, 24852): 9831,
 (47209, 13176): 19286,
 (47209, 21137): 12686,
 (47209, 21137, 24852): 2162,
 (47209, 21137, 13176): 4596,
 (47209, 21903): 10748,
 (47209, 21903, 24852): 1908,
 (47209, 21903, 13176): 3729,
 (47209, 21903, 21137): 2522,
 (47209, 21903, 21137, 13176): 1022,
 (47766,): 54730,
 (47766, 24852): 16419,
 (47766, 13176): 7078,
 (47766, 21137): 7375,
 (47766, 21137, 24852): 2734,
 (47766, 21137, 13176): 1410,
 (47766, 21903): 9553,
 (47766, 21903, 24852): 3284,
 (47766, 21903, 13176): 1550,
 (47766, 21903, 21137): 1560,
 (47626,): 47480,
 (47626, 24852): 12723,
 (47626, 13176): 6025,
 (47626, 21137): 5158,
 (47626, 21137, 24852): 1734,
 (47626, 21137, 13176): 1132,
 (47626, 21903): 7101,
 (47626, 21903

In [29]:
def get_support(T, frequency_table):
    df = pd.DataFrame((frequency_table.keys(), frequency_table.values())).T
    df.columns = ['itemset','support']
    df['support'] = df['support']/len(T)
    df['itemset'] = df['itemset'].apply(frozenset)
    return df

def get_association_rules(support_table, names):
    support_map = {k:v for k,v in support_table.values}
    index2item = {index:item for index, item in names.values}
    rules_table = []
    
    for itemset in support_map:
        if len(itemset) > 1:
            for item in itemset:
                antecedent = itemset - set([item])
                consequent = frozenset([item])
                
                confidence = support_map[itemset]/(support_map[antecedent])
                lift = support_map[itemset]/(support_map[antecedent]*support_map[consequent])
                
                antecedent_name = [index2item[index] for index in antecedent]
                consequent_name = [index2item[index] for index in consequent]
                rules_table.append([antecedent_name, consequent_name, confidence, lift])
                
    return pd.DataFrame(rules_table, columns = ['antecedant','consequent','confidence','lift'])

In [30]:
support_table = get_support(market_baskets_subset, frequency_table)
get_association_rules(support_table, products[['product_id','product_name']]).sort_values(by='lift', ascending = False)

,antecedant,consequent,confidence,lift
4435,[Grapefruit Sparkling Water],[Lemon Sparkling Water],0.223871,75.402771
4434,[Lemon Sparkling Water],[Grapefruit Sparkling Water],0.345571,75.402771
4260,[Icelandic Style Skyr Blueberry Non-fat Yogurt],[Non Fat Raspberry Yogurt],0.383120,74.133150
4261,[Non Fat Raspberry Yogurt],[Icelandic Style Skyr Blueberry Non-fat Yogurt],0.446207,74.133150
4437,[Icelandic Style Skyr Blueberry Non-fat Yogurt],[Non Fat Acai & Mixed Berries Yogurt],0.198704,72.732102
...,...,...,...,...
2704,[Bunched Cilantro],[Bag of Organic Bananas],0.080532,0.681231
2477,[Banana],[Hass Avocados],0.010672,0.679589
2476,[Hass Avocados],[Banana],0.100108,0.679589
2207,[Spring Water],[Banana],0.099321,0.674245


In [31]:
get_association_rules(support_table, products[['product_id','product_name']]).sort_values(by='lift', ascending = False).to_csv('association_rules.csv')